In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os 
os.chdir("/content/drive/MyDrive/Projects/BDD10K")

In [3]:
from glob import glob
all_data_img = glob("/content/drive/MyDrive/Projects/BDD10K/alldata/*.jpg")
len(all_data_img)

0

In [ ]:
all_txt_file = glob('/content/drive/MyDrive/Projects/BDD10K/alldata/*.txt')
len(all_txt_file)

In [ ]:
# This script will delete all the labels from the yolo labels except car 

import os
import glob
import shutil
from tqdm import tqdm 

# Path to the folder containing the yolo labels
path = 'alldata'

# Path to the folder where you want to save the modified labels
save_path = 'alldata'

# label code for car
car = 0

for filename in tqdm(os.listdir(path)):
    # continue if the file is classes.txt
    if filename == 'classes.txt':
        continue
    if filename.endswith(".txt"):
        with open(os.path.join(path, filename), 'r') as f:
            lines = f.readlines()
            f.close()
        with open(os.path.join(save_path, filename), 'w') as f:
            for line in lines:
                if int(line.split(' ')[0]) == car:
                    f.write(line)
            f.close()

100%|██████████| 20000/20000 [00:59<00:00, 337.52it/s]


In [ ]:
# This script will split the data into training and validation sets

import os
import random
import shutil
from glob import glob
from tqdm import tqdm

# creating folder for training and validation
if not os.path.exists('train'):
    os.makedirs('train')
if not os.path.exists('valid'):
    os.makedirs('valid')

# creading folder for images and labels
if not os.path.exists('train/images'):
    os.makedirs('train/images')
if not os.path.exists('train/labels'):
    os.makedirs('train/labels')

# Creating partitions of the data after shuffeling
# Get the list of all images
image_path_list = glob('alldata/*.jpg')
label_path_list = glob('alldata/*.txt')

print('number of images: ', len(image_path_list))
print('number of labels: ', len(label_path_list))

# Shuffle the images
random.shuffle(image_path_list)
random.shuffle(label_path_list)

# Calculate the split size
train_size = int(0.9 * len(image_path_list))
valid_size = int(0.1 * len(image_path_list))

# Copy-pasting images
train_image_list = image_path_list[:train_size]
valid_image_list = image_path_list[train_size:train_size+valid_size]

train_label_list = label_path_list[:train_size]
valid_label_list = label_path_list[train_size:train_size+valid_size]

# Train
for image_path in tqdm(train_image_list):
    shutil.copy(image_path, 'train/images')

for label_path in tqdm(train_label_list):
    shutil.copy(label_path, 'train/labels')

# Validation
for image_path in tqdm(valid_image_list):
    shutil.copy(image_path, 'valid/images')
    
for image_path in tqdm(valid_image_list):
    shutil.copy(image_path, 'valid/labels')

number of images:  10000
number of labels:  10000


100%|██████████| 1000/1000 [00:05<00:00, 175.71it/s]


# Preparing the dataset for training

In [ ]:
!pwd

/content/drive/MyDrive/Projects/BDD10K


In [ ]:
yaml_text = """train: ../train
val: ../valid

nc: 1
names: ['Car']"""

In [ ]:
with open("data.yaml", 'w') as f:
    f.write(yaml_text)
%cat data.yaml

train: ../train
val: ../valid

nc: 1
names: ['Car']

In [ ]:
# check the assigned GPU type
!nvidia-smi

Fri Nov 11 10:59:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#YOLO 
# !git clone https://github.com/ultralytics/yolov5  && cd yolov5 && pip install -r requirements.txt


In [ ]:
!python ./yolov5/train.py --img 512 --batch 16 --epochs 100 --workers 2 --data data.yaml --cfg "./yolov5/models/yolov5s.yaml" --name yolov5x_car --cache


train: weights=yolov5/yolov5s.pt, cfg=./yolov5/models/yolov5s.yaml, data=data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=yolov5/runs/train, name=yolov5x_car, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-237-g55e9516 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t